In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
import xgboost as xgb
import optuna

# Load the data
X_train_1 = np.load('/kaggle/input/lfd-kaggle/train_feats.npy', allow_pickle=True).item()
labels_df = pd.read_csv('/kaggle/input/lfd-kaggle/train_labels.csv')
y_train = labels_df['label'].values

X_train_resnet = X_train_1['resnet_feature']
X_train_vit = X_train_1['vit_feature']
X_train_clip = X_train_1['clip_feature']
X_train_dino = X_train_1['dino_feature']
X_train_combined = np.concatenate([X_train_resnet, X_train_vit, X_train_clip, X_train_dino], axis=1)

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Define the MLP model
class MLPFeatureExtractor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPFeatureExtractor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
       

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Train the MLP model
def train_mlp(X_train, y_train, input_size, hidden_size, output_size, epochs=10, batch_size=64, learning_rate=0.01):
    # Convert data to PyTorch tensors
    X_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_tensor = torch.tensor(y_train, dtype=torch.long)

    # Create a PyTorch dataset and DataLoader
    dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Initialize the model, loss, and optimizer
    model = MLPFeatureExtractor(input_size, hidden_size, output_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in data_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

    # Extract features
    model.eval()
    with torch.no_grad():
        X_transformed = model.fc1(X_tensor).numpy()  # Extract hidden-layer features

    return X_transformed, model

# Define hyperparameters for MLP
input_size = X_train_combined.shape[1]
hidden_size = 16
output_size = len(np.unique(y_train_encoded))

# Train the MLP and extract features
X_transformed, mlp_model = train_mlp(X_train_combined, y_train_encoded, input_size, hidden_size, output_size)

# Use transformed features for XGBoost
def objective_xgboost(trial):
    params = {
        'objective': 'multi:softmax',
        'num_class': len(np.unique(y_train_encoded)),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'eta': trial.suggest_loguniform('eta', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'lambda': trial.suggest_float('lambda', 0, 10),
        'alpha': trial.suggest_float('alpha', 0, 10),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'seed': 42,
        'tree_method': 'hist',  # Use GPU if available
        'device':'cuda'
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1_scores = []

    for train_idx, val_idx in skf.split(X_transformed, y_train_encoded):
        X_train, X_val = X_transformed[train_idx], X_transformed[val_idx]
        y_train, y_val = y_train_encoded[train_idx], y_train_encoded[val_idx]

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)

        model = xgb.train(params, dtrain, num_boost_round=trial.suggest_int('num_boost_round', 100, 500))

        y_val_pred = model.predict(dval)
        y_val_pred = np.round(y_val_pred).astype(int)

        f1 = f1_score(y_val, y_val_pred, average='macro')
        f1_scores.append(f1)

    return np.mean(f1_scores)

# Run Optuna for XGBoost
study = optuna.create_study(direction='maximize')
study.optimize(objective_xgboost, n_trials=10)

print("Best Parameters for XGBoost with MLP features:", study.best_params)


[I 2024-12-18 15:07:09,782] A new study created in memory with name: no-name-d0c48903-98f9-44ef-b592-71a7fb28cd9c
/tmp/ipykernel_23/3639997840.py:88: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.01, 0.3),
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [15:07:09] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [15:07:12] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [15:07:14] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warn

Best Parameters for XGBoost with MLP features: {'max_depth': 4, 'eta': 0.0802567270743116, 'subsample': 0.5879931167445216, 'colsample_bytree': 0.8373021840774447, 'gamma': 3.150304789281446, 'lambda': 8.142370366504458, 'alpha': 2.738733472635053, 'n_estimators': 223, 'num_boost_round': 392}


In [6]:
from xgboost import XGBClassifier

#best_params = {'max_depth': 8, 'eta': 0.027112223293722308, 'subsample': 0.911504725837794, 'colsample_bytree': 0.9632668796201045, 'gamma': 6.656675330872401, 'lambda': 7.6387053729149805, 'alpha': 5.958534201965637, 'n_estimators': 453, 'num_round': 107}
best_params = study.best_params

final_model = XGBClassifier(
    **best_params,
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42,
    tree_method="hist",
    device="cuda"
)
final_model.fit(X_transformed, y_train_encoded)

# Test Verisi ile Tahmin
X_valtest = np.load('/kaggle/input/lfd-kaggle/valtest_feats.npy', allow_pickle=True).item()
X_val_resnet = X_valtest['resnet_feature']
X_val_vit = X_valtest['vit_feature']
X_val_clip = X_valtest['clip_feature']
X_val_dino = X_valtest['dino_feature']

X_val_combined = np.concatenate([X_val_resnet, X_val_vit, X_val_clip, X_val_dino], axis=1)

# Convert test data to PyTorch tensors
X_val_tensor = torch.tensor(X_val_combined, dtype=torch.float32)
# Use the trained MLP to extract features
mlp_model.eval()  # Set the MLP model to evaluation mode
with torch.no_grad():
    X_val_transformed = mlp_model.fc1(X_val_tensor).numpy()  # Extract hidden-layer features

#dval = xgb.DMatrix(X_val_transformed)


predictions = final_model.predict(X_val_transformed)
y_val_pred_decoded = label_encoder.inverse_transform(predictions.astype(int))


# Tahminleri Kaydetme
prediction_df = pd.DataFrame({
    'ID': np.arange(len(y_val_pred_decoded)),
    'Predicted': y_val_pred_decoded
})
prediction_df.to_csv('xg_final.csv', index=False)

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [15:13:15] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)
